O Princípio do Aberto/Fechado (OCP) é o segundo princípio do SOLID e afirma que entidades de software (classes, módulos, funções, etc.) devem estar abertas para extensão, mas fechadas para modificação. Isso significa que você deve ser capaz de adicionar novas funcionalidades sem alterar o código existente. https://programadriano.medium.com/princ%C3%ADpios-do-solid-com-typescript-2c6a4911242d


In [14]:
public abstract class Investimento {
    protected String nome;
    protected Double valorInvestido;

    public Investimento(String nome, double valorInvestido) {
        this.nome = nome;
        this.valorInvestido = valorInvestido;
    }

    public abstract double calcularRetornoAnual();

    public String descrever() {
        return "Investimento: " + nome + ", Valor Investido: R$ " + valorInvestido;
    }
}

Classe Acao

In [15]:
public class Acao extends Investimento {
    private int quantidade;
    private double precoPorAcao;
    private double dividendoAnual;

    public Acao(String nome, double valorInvestido, int quantidade, double precoPorAcao, double dividendoAnual) {
        super(nome, valorInvestido);
        this.quantidade = quantidade;
        this.precoPorAcao = precoPorAcao;
        this.dividendoAnual = dividendoAnual;
    }

    @Override
    public double calcularRetornoAnual() {
        return quantidade * dividendoAnual;
    }
   

    public double buscarPrecoPorAcao() {
        return precoPorAcao;
    }
    
    public String getNome() {
        return nome;
    }

    public int buscarQuantidade(){
        return quantidade;
    }

    public double buscarDividendoAnual(){
        return quantidade;
    }

    @Override
    public String toString() {
        return "Acao{" +
                "nome='" + nome + '\'' +
                ", valorInvestido=" + valorInvestido +
                ", quantidade=" + quantidade +
                ", precoPorAcao=" + precoPorAcao +
                ", dividendoAnual=" + dividendoAnual +
                '}';
    }
}



Classe Titulo

In [16]:
public class Titulo extends Investimento {
    private double taxaDeJurosAnual;

    public Titulo(String nome, double valorInvestido, double taxaDeJurosAnual) {
        super(nome, valorInvestido);
        this.taxaDeJurosAnual = taxaDeJurosAnual;
    }

    @Override
    public double calcularRetornoAnual() {
        return this.valorInvestido * (this.taxaDeJurosAnual / 100);
    }
  
}

Classe Imovel

In [18]:
public class Imovel extends Investimento {
    private double aluguelMensal;

    public Imovel(String nome, double valorInvestido, double aluguelMensal) {
        super(nome, valorInvestido);
        this.aluguelMensal = aluguelMensal;
    }

    @Override
    public double calcularRetornoAnual() {
        return this.aluguelMensal * 12;
    }
 
}

InvestimentoService

In [20]:
public class InvestimentoService {
    
    public double calcularRetornoAnual(Investimento investimento) {
        if (investimento instanceof Acao) {
            Acao acao = (Acao) investimento;
            return acao.calcularRetornoAnual();
        } else if (investimento instanceof Titulo) {
            Titulo titulo = (Titulo) investimento;
            return titulo.calcularRetornoAnual();
        } else if (investimento instanceof Imovel) {
            Imovel imovel = (Imovel) investimento;
            return imovel.calcularRetornoAnual();
        }
        throw new IllegalArgumentException("Tipo de investimento desconhecido");
    }
   
}


In [27]:
Investimento acao = new Acao("Empresa X", 5000, 10, 50, 5);
Investimento titulo = new Titulo("Título do Governo", 10000, 5);
Investimento imovel = new Imovel("Apartamento no Centro", 300000, 2000);

InvestimentoService investimentoService = new InvestimentoService();

System.out.println("Retorno anual: R$ " + investimentoService.calcularRetornoAnual(acao));

System.out.println("Retorno anual: R$ " + investimentoService.calcularRetornoAnual(titulo));

System.out.println("Retorno anual: R$ " + investimentoService.calcularRetornoAnual(imovel));

Retorno anual: R$ 50.0
Retorno anual: R$ 500.0
Retorno anual: R$ 24000.0


Refatorando

In [29]:
public interface IInvestimentoService<Investimento> {
    double calcularRetornoAnual(investimento);
}

In [30]:
public class AcaoService implements IInvestimentoService<Acao> {
    @Override
    public double calcularRetornoAnual(Acao acao) {
        return acao.calcularRetornoAnual();
    }
}

In [31]:
public class TituloService implements IInvestimentoService<Titulo> {
    @Override
    public double calcularRetornoAnual(Titulo titulo) {
        return titulo.calcularRetornoAnual();
    }
}

In [32]:
public class ImovelService implements IInvestimentoService<Imovel> {
    @Override
    public double calcularRetornoAnual(Imovel imovel) {
        return imovel.calcularRetornoAnual();
    }
}

Registrando [DI]

In [35]:
public class Container {
    private Map<String, Class<?>> services = new HashMap<>();

    public <T> void register(String name, Class<T> implementation) {
        services.put(name, implementation);
    }

    public <T> T resolve(String name) {
        Class<?> implementation = services.get(name);
        if (implementation == null) {
            throw new RuntimeException("Service not found: " + name);
        }
        try {
            return (T) implementation.getDeclaredConstructor().newInstance();
        } catch (Exception e) {
            throw new RuntimeException("Failed to instantiate service: " + name, e);
        }
    }
}

Implementação

In [43]:
Container container = new Container();
container.register("AcaoService", AcaoService.class);
container.register("TituloService", TituloService.class);
container.register("ImovelService", ImovelService.class);

// Resolver serviços
IInvestimentoService<Acao> acaoService = container.resolve("AcaoService");
IInvestimentoService<Titulo> tituloService = container.resolve("TituloService");
IInvestimentoService<Imovel> imovelService = container.resolve("ImovelService");

// Criar instâncias de investimento
Acao acao = new Acao("Empresa X", 5000, 100, 50, 5);
Titulo titulo = new Titulo("Título do Governo", 10000, 10);
Imovel imovel = new Imovel("Apartamento no Centro", 300000, 1000);

// Usar os serviços
System.out.println("Retorno anual da ação: R$ " + acaoService.calcularRetornoAnual(acao));
System.out.println("Retorno anual do título: R$ " + tituloService.calcularRetornoAnual(titulo));
System.out.println("Retorno anual do imóvel: R$ " + imovelService.calcularRetornoAnual(imovel));

Retorno anual da ação: R$ 500.0
Retorno anual do título: R$ 1000.0
Retorno anual do imóvel: R$ 12000.0
